# Make each level to each Curve CSV FILE

In [65]:
import pandas as pd

In [273]:
df_begin = []
f_1 = 'beginner_expert_processedData/beginner/beginner_'
f_3 = '.csv'
num_begin = 19
curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]

df_concat = pd.DataFrame()
for curve_num in range(0,6):
    for idx in range(1, num_begin+1):
        tmp_file = f_1+str(idx)+'_new2'+f_3
        df = pd.read_csv(tmp_file)
        df = df.dropna()
        tmp = df.astype(float)
        tmp['level'] =0

        tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]

        df_begin.append(tmpcorner)
        df_concat = pd.concat([df_concat,df_begin[idx-1]])
    
    df_concat.to_csv('cornerData/corner_'+str(curve_num+1)+'_begin'+'.csv')
    df_concat = pd.DataFrame()

In [274]:
df_exp = []
f_1 = 'beginner_expert_processedData/expert/expert_'
f_3 = '.csv'
num_exp = 19

df_concat = pd.DataFrame()

for curve_num in range(0,6):
    for idx in range(1, num_exp+1):
        tmp_file = f_1+str(idx)+'_new2'+f_3
        df = pd.read_csv(tmp_file)
        df = df.dropna()

        tmp = df.astype(float)
        tmp['level'] =1

        tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]

        df_exp.append(tmpcorner)
        df_concat = pd.concat([df_concat,df_exp[idx-1]])
    
    df_concat.to_csv('cornerData/corner_'+str(curve_num+1)+'_expert'+'.csv')
    df_concat = pd.DataFrame()

In [336]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

df_tmp_begin = pd.read_csv('cornerData/corner_1_begin.csv')
df_tmp_exp   = pd.read_csv('cornerData/corner_1_expert.csv')

df_curve1 = pd.concat([df_tmp_begin, df_tmp_exp])

left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration','level']


df_curve1= df_curve1.loc[:,left_column]
df_curve1_saved = df_curve1.loc[:,left_column]

y = df_curve1.pop('level')
X = df_curve1

y = np.array(y)
X = np.array(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

svm_model = SVC(kernel='rbf', C=1, gamma=0.01)
 
svm_model.fit(X_train_std, y_train)
y_pred = svm_model.predict(X_test_std)
print(svm_model.score(X_train_std,y_train))
print(svm_model.score(X_test_std,y_test))


scoreSet = cross_val_score(svm_model, X_test_std, y_test, cv=5)
print("\n\rUsing model_selection library    ")
print("score set : ",end="")
print(scoreSet)


1.0
0.9946332737030411

Using model_selection library    
score set : [0.97321429 0.96428571 0.95535714 0.98214286 0.93693694]


In [292]:
from sklearn.model_selection import GridSearchCV
param={'C':[0.1,1,10,100,1000],
      'gamma':[0.01,0.1,1,10,100]}
GS=GridSearchCV(SVC(kernel='linear'),param, cv=5)
GS.fit(X_train_std, y_train)
print(GS.best_params_)
print(GS.best_score_)

{'C': 1, 'gamma': 0.01}
0.9995525727069351


In [294]:
# 분류 성능 평가 지표 생성
import sklearn.metrics as mt

actual_result = y_test
predicted_result = y_pred
# 정확도
accuracy = mt.accuracy_score(actual_result, predicted_result)
# 오차행렬
matrix = mt.confusion_matrix(actual_result, predicted_result)
# 정밀도
precision = mt.precision_score(actual_result, predicted_result)
# 재현율
recall = mt.recall_score(actual_result, predicted_result)
# f1 score
f1score = mt.f1_score(actual_result, predicted_result)
# 전체보고서
scores = mt.classification_report(actual_result, predicted_result)
# 모두 출력
print("정확도: ", accuracy, "\n")
print("오차행렬\n", matrix, "\n")
print("정밀도: ", precision, "\n")
print("재현율: ", round(recall, 3), "\n")
print("F1스코어: ", f1score, "\n")
print("결과\n", scores)


정확도:  0.9821109123434705 

오차행렬
 [[303   1]
 [  9 246]] 

정밀도:  0.9959514170040485 

재현율:  0.965 

F1스코어:  0.9800796812749004 

결과
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       304
           1       1.00      0.96      0.98       255

    accuracy                           0.98       559
   macro avg       0.98      0.98      0.98       559
weighted avg       0.98      0.98      0.98       559



In [252]:
!pip install Boruta

In [295]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute

df_tmp_begin = pd.read_csv('cornerData/corner_1_begin.csv')
df_tmp_exp   = pd.read_csv('cornerData/corner_1_expert.csv')
df_curve1 = pd.concat([df_tmp_begin, df_tmp_exp])
df_curve1= df_curve1.loc[:,left_column]


y = df_curve1.pop('level')
X = df_curve1

y = np.array(y)
X = np.array(X)


# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features - 5 features should be selected
feat_selector.fit(X, y)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	102
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	73
Tentative: 	29
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	73
Tentative: 	25
Rejected: 	4
Iteration: 	10 / 100
Confirmed: 	73
Tentative: 	25
Rejected: 	4
Iteration: 	11 / 100
Confirmed: 	73
Tentative: 	25
Rejected: 	4
Iteration: 	12 / 100
Confirmed: 	75
Tentative: 	23
Rejected: 	4
Iteration: 	13 / 100
Confirmed: 	75
Tentative: 	23
Rejected: 	4
Iteration: 	14 / 100
Confirmed: 	75
Tentative: 	23
Rejected: 	4
Iteration: 	15 / 100
Confirmed: 	75
Tentative: 	23
Rejected: 	4
Iteration: 	16 / 100
Confirmed: 	79
Tentative: 	1

In [317]:
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y, test_size = 0.2, stratify = y)
 
sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

svm_model = SVC(kernel='rbf', C=1, gamma=0.01)
 
svm_model.fit(X_train_std, y_train)
y_pred = svm_model.predict(X_test_std)
print(svm_model.score(X_train_std,y_train))
print(svm_model.score(X_test_std,y_test))

actual_result = y_test
predicted_result = y_pred
# 정확도
accuracy = mt.accuracy_score(actual_result, predicted_result)
# 오차행렬
matrix = mt.confusion_matrix(actual_result, predicted_result)
# 정밀도
precision = mt.precision_score(actual_result, predicted_result)
# 재현율
recall = mt.recall_score(actual_result, predicted_result)
# f1 score
f1score = mt.f1_score(actual_result, predicted_result)
# 전체보고서
scores = mt.classification_report(actual_result, predicted_result)
# 모두 출력
print("정확도: ", accuracy, "\n")
print("오차행렬\n", matrix, "\n")
print("정밀도: ", precision, "\n")
print("재현율: ", round(recall, 3), "\n")
print("F1스코어: ", f1score, "\n")

print("결과\n", scores)

1.0
0.9964221824686941
정확도:  0.9964221824686941 

오차행렬
 [[303   1]
 [  1 254]] 

정밀도:  0.996078431372549 

재현율:  0.996 

F1스코어:  0.996078431372549 

결과
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       304
           1       1.00      1.00      1.00       255

    accuracy                           1.00       559
   macro avg       1.00      1.00      1.00       559
weighted avg       1.00      1.00      1.00       559



In [339]:
from sklearn.model_selection import KFold

X = ["a","b","c","d"]
kf = KFold(n_splits=2)
kf.split(X)

<generator object _BaseKFold.split at 0x0000018F1E2AE510>